In [137]:
from pandas import Series, DataFrame
from snownlp import SnowNLP
import numpy as np
import jieba
import jieba.analyse

In [2]:
import pandas as pd
df = pd.read_csv('C:/Users/Robin/Documents/Code/Data/shenjianshou/jd_logi_reviews.csv', encoding='utf-8')
df2 = pd.read_csv('C:/Users/Robin/Documents/Code/Data/shenjianshou/jd_logi_products.csv', encoding='utf-8')

FileNotFoundError: File b'C:/Users/Robin/Documents/Code/Data/shenjianshou/jd_logi_reviews.csv' does not exist

In [146]:
df2.head(3)

,爬取链接(__url),爬取时间(__time),product_id,name,商品参数(params),current_price,是否自营(is_self_run),品牌(brand),商品分类(categories),店铺ID(shop_id),店铺名称(shop_name),商品链接URL(url),商品描述(description),score,comments_count
0,http://item.jd.com/10475125646.html,6/27/2017 1:48,10475125646,罗技（Logitech）CC2000e 商务高清视频会议摄像头 Conference,"<li title=""罗技（Logitech）CC2000e 商务高清视频会议摄像头 Con...",3199.0,否,罗技（Logitech）,"[""电脑、办公"",""外设产品"",""摄像头""]",118944,罗技融合专卖店,http://item.jd.com/10475125646.html,【下单送MK345键鼠套装】一到六人，便携式视频会议系统，发声麦克风！,100,1.0
1,http://item.jd.com/13350570400.html,6/27/2017 1:48,13350570400,罗技（Logitech）优联接收器 支持连接优联技术的罗技无线鼠标键盘接收器 键鼠优联接收器,"<li title=""罗技（Logitech）优联接收器 支持连接优联技术的罗技无线鼠标键盘...",39.9,否,罗技（Logitech）,"[""电脑、办公"",""外设产品"",""鼠标""]",118944,罗技融合专卖店,http://item.jd.com/13350570400.html,NaN,100,NaN
2,http://item.jd.com/12319422586.html,6/27/2017 1:48,12319422586,捷伯德（Jaybird）Freedom Wireless 无线蓝牙运动耳机 跑步防水入耳式 蓝色,"<li title=""捷伯德（Jaybird）Freedom Wireless 无线蓝牙运动...",1199.0,否,JAYBIRD,"[""数码"",""影音娱乐"",""耳机/耳麦"",""JAYBIRD""]",118944,罗技融合专卖店,http://item.jd.com/12319422586.html,新品上架，立享促销价1199！,100,NaN


In [7]:
best_products = df2.sort_values('comments_count',axis=0, ascending=False)[0:50]
pro_ids = best_products['product_id'].values

In [175]:
senti_mean = []
pro_id_new = []
for pro_id in pro_ids:
    review_sentiments = []
    reviews = df[(df.product_id == pro_id) & (df.score <= 5)]['content']
    if reviews.empty:
        result_mean = 0
    else:
        for review in reviews:
            try:
                s = SnowNLP(str(review))
                review_sentiment =s.sentiments
                review_sentiments.append(review_sentiment)
            except:
                review_sentiments.append('NaN')
        review_obj = DataFrame({'review':reviews,
                                'sentiment':review_sentiments})

        result_mean = review_obj['sentiment'].mean(axis=0)
        pro_id_new.append(pro_id) 
        senti_mean.append(result_mean)
# df_senti = DataFrame({'pro_ids':pro_ids,
#                         'sentiment':senti_mean})

In [177]:
senti_series = pd.DataFrame({'senti':senti_mean,'product_id':pro_id_new})
best_pro_new = best_products.merge(senti_series, on='product_id')
best_pro_new.head(5)

,爬取链接(__url),爬取时间(__time),product_id,name,商品参数(params),current_price,是否自营(is_self_run),品牌(brand),商品分类(categories),店铺ID(shop_id),店铺名称(shop_name),商品链接URL(url),商品描述(description),score,comments_count,senti
0,http://item.jd.com/1439105.html,6/27/2017 1:23,1439105,罗技（Logitech）K120键盘+M105 鼠标（炫酷黑）键鼠套装,"<li title=""罗技K120键盘+M105 鼠标（炫酷黑）升级版"">商品名称：罗技K1...",98.0,是,罗技（Logitech）,"[""电脑、办公"",""外设产品"",""键盘""]",1000000248,罗技官方旗舰店,http://item.jd.com/1439105.html,NaN,96,344315.0,0.788119
1,http://item.jd.com/262214.html,6/27/2017 1:20,262214,罗技（Logitech）K120键盘,"<li title=""罗技K120"">商品名称：罗技K120</li> <li tit...",45.0,是,罗技（Logitech）,"[""电脑、办公"",""外设产品"",""键盘""]",1000000248,罗技官方旗舰店,http://item.jd.com/262214.html,"罗技经典，高性价比键盘，持久耐用,防水溅设计!",97,301703.0,0.779168
2,http://item.jd.com/423478.html,6/27/2017 1:20,423478,罗技（Logitech）M185 无线鼠标 黑色灰边,"<li title=""罗技M185"">商品名称：罗技M185</li> <li tit...",69.0,是,罗技（Logitech）,"[""电脑、办公"",""外设产品"",""鼠标""]",1000000248,罗技官方旗舰店,http://item.jd.com/423478.html,"高性价比,造型小巧,便于携带,人体工程学！",97,284459.0,0.728385
3,http://item.jd.com/584300.html,6/27/2017 1:20,584300,罗技（Logitech）MK120 键鼠套装 鼠标键盘套装 黑色,"<li title=""罗技MK120"">商品名称：罗技MK120</li> <li t...",75.0,是,罗技（Logitech）,"[""电脑、办公"",""外设产品"",""键盘""]",1000000248,罗技官方旗舰店,http://item.jd.com/584300.html,罗技经典有线套装！,97,249562.0,0.761891
4,http://item.jd.com/692918.html,6/27/2017 1:20,692918,罗技（Logitech）M100r 光电鼠标 （黑色）,"<li title=""罗技M100r"">商品名称：罗技M100r</li> <li t...",44.0,是,罗技（Logitech）,"[""电脑、办公"",""外设产品"",""鼠标""]",1000000248,罗技官方旗舰店,http://item.jd.com/692918.html,罗技畅销有线鼠标，黑白双色，全尺寸设计，外形精巧！,97,234962.0,0.746239


In [183]:
best_pro_extract = pd.DataFrame({'name':best_pro_new.name,
                                'id':best_pro_new.product_id,
                                'price':best_pro_new.current_price,
                                'senti':best_pro_new.senti,
                                 'score':best_pro_new.score,
                                'reviews':best_pro_new.comments_count
                                })
best_pro_extract.head(50)

,id,name,price,reviews,score,senti
0,1439105,罗技（Logitech）K120键盘+M105 鼠标（炫酷黑）键鼠套装,98.0,344315.0,96,0.788119
1,262214,罗技（Logitech）K120键盘,45.0,301703.0,97,0.779168
2,423478,罗技（Logitech）M185 无线鼠标 黑色灰边,69.0,284459.0,97,0.728385
3,584300,罗技（Logitech）MK120 键鼠套装 鼠标键盘套装 黑色,75.0,249562.0,97,0.761891
4,692918,罗技（Logitech）M100r 光电鼠标 （黑色）,44.0,234962.0,97,0.746239
5,1439133,罗技（Logitech）M275 无线鼠标（黑色）+K230 无线键盘 键鼠套装,188.0,182150.0,97,0.884772
6,1208740,罗技（Logitech）M275 无线鼠标 黑色,99.0,158876.0,97,0.764084
7,276034,罗技（Logitech）C270 高清网络摄像头,139.0,147210.0,96,0.722985
8,852431,罗技（Logitech） M545 无线鼠标 黑色,149.0,79840.0,95,0.722589
9,4427554,罗技（Logitech）G302+G610茶轴 MOBA游戏套装 键鼠游戏套装 键盘鼠标,1048.0,56752.0,95,0.698125


In [6]:
review_sentiments = []
for review in reviews:
    try:
        s = SnowNLP(str(review))
        review_sentiment =s.sentiments
        review_sentiments.append(review_sentiment)
    except:
        review_sentiments.append('NaN')
review_obj = DataFrame({'review':reviews,
                        'sentiment':review_sentiments})

In [7]:
print (review_obj.sort_values('sentiment',axis=0)[0:10])

                                                  review     sentiment
212    这是我在京东第一次遇到如此不靠谱的客服，收货后尝试连接了好几次都无法正常使用，但电脑却未提示...  0.000000e+00
15225  给一分评价是给京东快递和换货的速度。强烈建议不要买这个鼠标，价格贵还带有先天不足的质量设计缺...  0.000000e+00
43741  商品不错，物流零分，14号到配送站，不送，15号回到分拨中心，15号下午又到同一配送站，又不...  1.776357e-15
12152  优点明显的同时，缺点也明显。 支持在mac和win间切换，定位精确，这个还是挺赞的，but！...  9.194503e-10
18608  严重差评！包装序列号和鼠标本身序列号不一致！！连保修问题都弄不清，还要我提供自己鼠标序列号登...  5.319390e-09
10686  第一次两星评价，原因以下几点：1）封塑包装拆开后，手托磨砂面 三面都有磨损，感觉是二手鼠标重...  2.235017e-08
5469   差评，我一口气买了7个、连个包装盒都没有就邮过来了！拆开都有磨损的，明显挤压的，这简直太差了...  3.004785e-07
18604  在win7试用还可以。在mac用完全扯淡&hellip;&hellip;滚轮拨一两格界面根本...  6.998234e-06
2685               刚拿到试了U这个键声音，手感和其他键不一样，已申请换货，出厂质量把关太差！  1.084785e-05
8475         差，蓝牙严重影响其他设备，甚至蓝牙音响，我的蓝牙鼠标，搞了好久才搞清楚，买的人千万慎重  1.622462e-05


In [8]:
review_obj.mean(axis=0)

sentiment    0.176329
dtype: float64

In [21]:
jieba.analyse.set_stop_words('C:/Users/Yanbo/Google Drive/Code/Project/TextMining/stoplist.txt')
jieba_text = reviews.tolist()
tags = jieba.analyse.extract_tags(str(jieba_text), topK=10, withWeight=True)
word_rank = pd.DataFrame(tags,columns=['word','rank'])
word_rank                                                                                              

,word,rank
0,鼠标,0.227467
1,罗技,0.224220
2,蓝牙,0.153953
3,京东,0.125607
4,键盘,0.100150
5,连接,0.072904
6,差评,0.065625
7,问题,0.058020
8,包装,0.048057
9,客服,0.047591


In [22]:
snow_reviews = ''
for review_temp in reviews:
    snow_reviews = snow_reviews + '。' + review_temp
s = SnowNLP(snow_reviews)
s.summary(20)

['强烈建议不要买这个鼠标',
 '这都不会做',
 '客服根本都讲不清楚怎么使用的而且使用起来并没有那么好用连接的速度也慢打字总是慢半拍  罗技的鼠标也买过一次99的可以说地摊货都比这个好呀   朋友们买的时候一定要注意呀 广告打的好并没有什么卵用  不信可以网上查查口碑差的品牌怎么会做到现在真是不理解 说多了都是泪',
 '手感也不是很好',
 '不知道是个案还是都这样',
 '鼠标刚开始手感很好',
 '你们罗技什么都不要卖',
 '和客服联系后说这种情况不能退货',
 '鼠标脚垫有痕迹这个就不说了',
 '使用起来不是很方便',
 '蓝牙连接总是掉线不知道什么问题',
 '我不动鼠标',
 '不好用',
 '滚轮就不好用了',
 '不让还会再对比一下其他',
 '一点都不好玩除了蓝牙都是噱头',
 '不让说啊',
 '看能不能归到差评',
 '不建议买',
 '与thinkpad x220蓝牙连接不上']

很奇怪，收到的包装竟然贴了双层的封贴，里面一层是打开过的。感觉像是二手的，京东的货现在真的让人担心。
这是我在京东第一次遇到如此不靠谱的客服，收货后尝试连接了好几次都无法正常使用，但电脑却未提示连接异常，先是在线咨询回复很慢，高峰时间能够理解，但是我话还没说完，客服已经着急让我做评价了，任何有用的指导都没有，包装内没有说明书也就算了，让客服发个操作步骤，也没有，好吧毕竟是在线的，沟通有所不畅也忍了，要了400客服电话，先是一个普通话极其不标准勉强能够听懂他说什么的客服，问了问题查办天也无回复，只能电话在线等，过了一会换个男士接电话，我告诉他我已经按照正常的连接方法连接了好几次了，电脑显示连接成功但还是没有办法使用键盘，先是质疑我的连接方式，我给他重复了至少两遍我是怎么操作的，他说我的操作正确，但是还是不行，我询问他是否连接正常后指示灯会亮，沉默无回答，然后说我的电脑没有蓝牙，我告诉他我每天都用蓝牙音响听歌，然后他问我是蓝牙吗？请问我连自己的电脑有没有蓝牙也不知道吗？要求与客服经理通话，这位先生慢悠悠的说对不起我就是经理，没有其他人了，言外之意就是你也只能找我说了，没有其他人会管你了，然后电话就莫名其妙的断了，再也打不进去，再打就是听完语音直接挂断，请问京东你们就是这么纵容入住的品牌的吗？
还好吧，开始送来的是一个二次包装销售的，最后还是给换了个，虽然一开始还不想给我换的，买的新的自然要送来新的，送来旧的怎么可以呢？
一个塑料袋就发来了，包装都压坏了。还怎么送人？
1. 连接win7+ x220，经常丢失信号，同时买的另一个k480也是这样的问题，是技术不成熟，还是罗技你的技术有问题？2. 同样的键盘我在某大厂z家买了一个k380+480，人家是用纸盒包装，你用塑料袋直接装。你看看你那键盘的包装都被挤压成啥样子了？如果是买来送女生还能送吗？3. k480的电池盖松动的这么厉害，到底是特供京东的有这种情况，还是罗技普遍这个情况？如果没有说明，我会认为两者兼有。而罗技，这个做了几十年的外设大厂，如果继续这样垃圾设计品控垃圾细节，那么距离倒闭也不远了。4. 一星给快递员。风里来雨里去，真心不容易。差评。
差评，我一口气买了7个、连个包装盒都没有就邮过来了！拆开都有磨损的，明显挤压的，这简直太差了，差评！店主都吝啬的不给包装盒了
没有任何保护措施，包装被挤压，导致鼠标表面有刮

AttributeError: 'str' object has no attribute 'name'